In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../")

import pickle
import time

import torch

import matplotlib.pyplot as plt
import defmod as dm
import model_graddesc as model

torch.set_default_tensor_type(torch.DoubleTensor)


In [25]:
gd = torch.tensor([0.,0.,1.,1.,2.,2.,3.,3.,4.,4.]).requires_grad_()


n = 5
#gd = torch.rand(2,n).requires_grad_()


man = dm.manifold.Landmarks(2,n, gd=gd.view(-1))
comp1 = dm.deformationmodules.CompoundModule([dm.deformationmodules.Translations(man, sigma=1.)])
globaltrans = dm.deformationmodules.GlobalTranslation(man, sigma=1.)
localscal = dm.deformationmodules.LocalScaling(man, sigma=1.)
localrot = dm.deformationmodules.LocalRotation(man, sigma=1.)

comp2 = dm.deformationmodules.CompoundModule([globaltrans])

modules = dm.multishape.MultiShapeModule([comp1, comp2], 0.5, reduce_background=False)
modules.fill_controls_zero()
constr_Id = dm.constraints.Identity_Silent()
#constr_Id = dm.constraints.Identity_Silent_reduced()
H = dm.hamiltonian_multishape.Hamiltonian_multi(modules, constr_Id)

# set parameters
reg_param = 10**-3 

gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
mom0_tensor = torch.cat(H.module.manifold.unroll_cotan().copy(),0).requires_grad_().view(-1)
    
gd0_tensor.retain_grad()
mom0_tensor.retain_grad()
    
#EnergyFunctional = model.EnergyFunctional(modules, H, constr_Id,source, target, 2, reg_param)

In [ ]:
def test_gradcheck_autoaction():
    def autoaction(gd, mom):
        mod.manifold.fill_gd(gd)
        mod.manifold.fill_cotan(mom)
        return mod.autoaction()
    return torch.autograd.gradcheck(autoaction, (gd, mom), raise_exception=True)

mod = modules
gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(mom0_tensor.shape)

test_gradcheck_autoaction()

In [ ]:
def test_gradcheck_fields():
    def autoaction(gd, mom):
        mod.manifold.fill_gd(gd)
        mod.manifold.fill_cotan(mom)
        points = torch.tensor([0.,0.]).view(-1,2)
        return mod.field_generator().fields[0](points)
    return torch.autograd.gradcheck(autoaction, (gd, mom), raise_exception=True)

mod = modules
gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(mom0_tensor.shape)

test_gradcheck_fields()

In [ ]:
def test_gradcheck_fields():
    def autoaction(gd, mom):
        mod.manifold.fill_gd(gd)
        mod.manifold.fill_cotan(mom)
        points = torch.tensor([0.,0.]).view(-1,2)
        #return mod.background.field_generator()(points)
        return mod.field_generator().fields[-1](points)
    return torch.autograd.gradcheck(autoaction, (gd, mom), raise_exception=True)

mod = modules
gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(mom0_tensor.shape)

test_gradcheck_fields()

In [ ]:
def test_gradcheck_fields():
    def autoaction(gd, mom):
        mod.manifold.fill_gd(gd)
        mod.manifold.fill_cotan(mom)
        
        fields = mod.field_generator().fieldlist

        gd_bg = mod.manifold.manifold_list[-1].gd_points()[0]
        
        return fields[0](gd_bg)
    
    return torch.autograd.gradcheck(autoaction, (gd, mom), raise_exception=True)

mod = modules
gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(mom0_tensor.shape)

test_gradcheck_fields()

In [ ]:
def test_gradcheck_geocont_from_self():
    def fun(gd, mom):
        mod.manifold.fill_gd(gd)
        mod.manifold.fill_cotan(mom)
        
        man = mod.manifold
        vs = mod.adjoint(man)
        K_q = dm.kernels.K_xx(mod.manifold.gd.view(-1, mod.manifold.dim), mod.sigma)
        x = vs(mod.manifold.gd.view(-1, mod.manifold.dim))
        controls, _ = torch.gesv(x, K_q)
        mod.fill_controls(controls.contiguous().view(-1))
        
        mod.compute_geodesic_control_from_self(mod.manifold)
        return mod.controls
    return torch.autograd.gradcheck(fun, (gd, mom), raise_exception=True)

mod = dm.deformationmodules.Translations(man, sigma=1.)
#mod = H.module

gd0_tensor = torch.cat(mod.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(gd.shape).requires_grad_()

test_gradcheck_geocont_from_self()

In [34]:
def test_gradcheck_H_geodesic_controls():
    def fun(gd, mom):
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        H.geodesic_controls()
        
        return torch.cat([*H.module.controls[0], *H.module.controls[1], *H.module.controls[-1]],0)
        #return H.constraints(H.module)
    
    return torch.autograd.gradcheck(fun, (gd, mom), raise_exception=True)

constr = dm.constraints.Identity_Silent()

mod = modules
gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(mom0_tensor.shape)

test_gradcheck_H_geodesic_controls()

True

In [ ]:
def test_gradcheck_apply_constr():
    def hamiltonian(gd, mom):
        
        
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        #H.geodesic_controls()
        
        modules.manifold.fill_gd(gd)
        modules.manifold.fill_cotan(mom)
               
        appl_constr = torch.dot(H.module.l.view(-1,1).squeeze(), constr(H.module).view(-1,1).squeeze())
        #return appl_constr
        return H.apply_constr()    
    
    return torch.autograd.gradcheck(hamiltonian, (gd, mom), raise_exception=True)

gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(mom0_tensor.shape).requires_grad_()

constr = dm.constraints.Identity_Silent()

test_gradcheck_apply_constr()

In [ ]:
def test_gradcheck_apply_mom():
    def hamiltonian(gd, mom):
        
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        #H.geodesic_controls()
               
        appl_mom = H.appl_mom()
        return apply_mom
            
    
    return torch.autograd.gradcheck(hamiltonian, (gd, mom), raise_exception=True)

gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor

constr = dm.constraints.Identity_Silent()

test_gradcheck_apply_constr()

In [29]:
def test_gradcheck_H_applconstr():
    def fun(gd, mom):
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        H.geodesic_controls()
        
        x = H.module.l.view(-1,1).squeeze()
        y = H.constraints(H.module).view(-1,1).squeeze()
        appl_constr = torch.dot(x, y)

        return x, y#torch.dot(x,y)
    
    return torch.autograd.gradcheck(fun, (gd, mom), raise_exception=True)


gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(mom0_tensor.shape)

test_gradcheck_H_applconstr()

True

In [31]:
def test_gradcheck_Hamiltonian():
    def hamiltonian(gd, mom):
        
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        H.geodesic_controls()
                
        #return H.apply_mom()
        return H.apply_constr()
    
    return torch.autograd.gradcheck(hamiltonian, (gd, mom), raise_exception=True)

gd0_tensor = torch.cat(H.module.manifold.unroll_gd().copy(),0).requires_grad_().view(-1)
gd = gd0_tensor
mom = 1000*torch.rand(gd.shape)


test_gradcheck_Hamiltonian()

RuntimeError: Jacobian mismatch for output 0 with respect to input 0,
numerical:tensor([[ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [-3.6446e+05],
        [-3.6446e+05],
        [-2.3162e+06],
        [-2.3162e+06],
        [ 1.4817e+06],
        [ 1.4817e+06],
        [ 1.2093e+06],
        [ 1.2093e+06],
        [-1.0346e+04],
        [-1.0346e+04],
        [-1.1017e+05],
        [-1.1017e+05],
        [ 3.0324e+05],
        [ 3.0324e+05],
        [-2.9104e-03],
        [-2.9104e-03],
        [-2.8315e+05],
        [-2.8315e+05],
        [ 7.5670e+04],
        [ 7.5670e+04],
        [ 1.0962e+05],
        [ 1.0962e+05],
        [-3.4819e+05],
        [-3.4819e+05],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 3.2809e+05],
        [ 3.2809e+05],
        [-7.5121e+04],
        [-7.5121e+04]])